# 🧩 Docker Microservices Security Audit Notebook

**Author:** Maks Zaikin  
**Repository:** [maxzaikin/SecOps](https://github.com/maxzaikin/SecOps/tree/main/docker-audit)  
**Date:** {{11-13-2025}}  

---

## ⚙️ Overview
This Jupyter Notebook contains a structured approach to auditing the security posture of containerized microservice architectures.  
It demonstrates how to collect, analyze, and correlate information from Docker environments to identify potential configuration and security issues.

---

## ⚠️ Disclaimer
This notebook is provided **as is**, without any warranty or guarantee of fitness for your environment.  
Use it **at your own risk** — always test in a **controlled or isolated environment** before applying any method in production systems.

---

> 🛡️ **Note:**  
> The goal of this notebook is educational and methodological — to share a reproducible audit approach, not to expose or assess any specific commercial product.


In [ ]:
import pandas as pd
import json

In [ ]:
def expand_network_data(df): 
    '''
    Description: Flatten json data
                 How to prepare data
                 1. Inpect network docker network inspect
                 2. Create a *.json file in VS code and place []
                 3. r.m. click Select 'Format Document'
    Return: 
    '''
    
    expanded_rows = [] 
    
    for _, row in df.iterrows(): 
        # Basic fields
        network_base = { 
            'network_name': row.get('Name', ''), 
            'network_id': row.get('Id', ''), 
            'network_created': row.get('Created', ''), 
            'network_scope': row.get('Scope', ''), 
            'network_driver': row.get('Driver', ''), 
            'network_is_ipv6_enabled': row.get('EnableIPv6', False), 
            'network_is_internal': row.get('Internal', False), 
            'network_is_attachable': row.get('Attachable', False), 
            'network_is_ingress': row.get('Ingress', False), 
            'netwrok_is_config_only': row.get('ConfigOnly', False) 
        } 

        # Explode IPAM 
        ipam = row.get('IPAM', {}) 
        
        network_base['network_ipam_driver'] = ipam.get('Driver', '') 
        network_base['network_ipam_options'] = ipam.get('Options', None) 

        # Explode IPAM.Config 
        ipam_config = ipam.get('Config', [{}]) 
        
        if ipam_config and len(ipam_config) > 0:
            network_base['netwrok_ipam_config_subnet'] = ipam_config[0].get('Subnet', '') 
            network_base['netwrok_ipam_config_gateway'] = ipam_config[0].get('Gateway', '') 
        else: 
            network_base['netwrok_ipam_config_subnet'] = '' 
            network_base['netwrok_ipam_config_gateway'] = '' 

        # Explode ConfigFrom 
        config_from = row.get('ConfigFrom', {}) 
        network_base['netwrok_config_from'] = config_from.get('Network', '') 

        # Обрабатываем Options и Labels 
        network_base['network_options'] = str(row.get('Options', {})) 
        network_base['netwrok_labels'] = str(row.get('Labels', {})) 

        # Process containers
        containers = row.get('Containers', {}) 
        
        if containers:
            
            for container_id, container_info in containers.items(): 
                expanded_row = network_base.copy() 
                expanded_row.update({'container_id': container_id, 
                                     'container_name': container_info.get('Name', ''), 
                                     'container_endpoint_id': container_info.get('EndpointID', ''), 
                                     'container_mac': container_info.get('MacAddress', ''), 
                                     'container_ipv4': container_info.get('IPv4Address', ''), 
                                     'container_ipv6': container_info.get('IPv6Address', '') }) 
                
                expanded_rows.append(expanded_row) 
        else: 
            # If there is no container add only network info
            expanded_row = network_base.copy() 
            expanded_row.update({'container_id': None, 
                                 'container_name': None, 
                                 'container_endpoint_id': None, 
                                 'container_mac': None, 
                                 'container_ipv4': None, 
                                 'container_ipv6': None }) 
            
            expanded_rows.append(expanded_row) 

    # Create final dataframe
    columns = ['network_name', 
               'network_id', 
               'network_created', 
               'network_scope', 
               'network_driver', 
               'network_is_ipv6_enabled', 
               'network_ipam_driver', 
               'network_ipam_options', 
               'netwrok_ipam_config_subnet', 
               'netwrok_ipam_config_gateway', 
               'network_is_internal', 
               'network_is_attachable', 
               'network_is_ingress', 
               'netwrok_config_from', 
               'netwrok_is_config_only', 
               'network_options', 
               'network_labels', 
               'container_id', 
               'container_name', 
               'container_endpoint_id', 
               'container_mac', 
               'container_ipv4', 
               'container_ipv6' 
              ] 
            
    return pd.DataFrame(expanded_rows, columns=columns) 


def expand_container_data(df: pd.DataFrame):
    """
    Description: Flatten `docker ps --no-trunc` JSON dump
                 How to prepare data
                 1. Run command: docker ps --no-trunc --format '{{json .}}'
                 2. Create a *.json file in VS code and place []
                 3. In between [] paste output 
                 4. r.m. click Select 'Format Document'
                 5.Don't forget to place ',' between {container data},{containcer data}... 
    Return: pandas.DataFrame
    
    """
    expanded_rows = []

    for _, row in df.iterrows():
        base = {
            'container_id': row.get('ID', ''),
            'container_name': row.get('Names', ''),
            'container_image': row.get('Image', ''),
            'container_command': row.get('Command', ''),
            'container_created_at': row.get('RunningFor', ''),
            'container_status': row.get('Status', ''),
            'container_ports_raw': row.get('Ports', ''),
            'container_labels_raw': row.get('Labels', ''),
            'container_mounts': row.get('Mounts', ''),
            'container_networks': row.get('Networks', ''),
            'container_local_names': row.get('LocalVolumes', ''),
            'container_state': row.get('State', ''),
            'container_size': row.get('Size', ''),
        }

        # ports parsing
        ports = row.get('Ports', '')
        if ports:
            base['container_ports'] = [p.strip() for p in ports.split(',')]
        else:
            base['container_ports'] = []

        # Парсим метки (Labels могут быть строкой или словарём)
        labels = row.get('Labels', '')
        if isinstance(labels, str) and '=' in labels:
            label_dict = {}
            for item in labels.split(','):
                if '=' in item:
                    k, v = item.split('=', 1)
                    label_dict[k.strip()] = v.strip()
            base['container_labels'] = label_dict
        elif isinstance(labels, dict):
            base['container_labels'] = labels
        else:
            base['container_labels'] = {}

        # explode everythinf in separate fields
        for k, v in base['container_labels'].items():
            safe_key = 'container_label_' + k.replace('.', '_').replace('-', '_')
            base[safe_key] = v

        expanded_rows.append(base)

    # Arrange cols
    cols = [
        'container_id',
        'container_name',
        'container_image',
        'container_command',
        'container_created_at',
        'container_status',
        'container_state',
        'container_networks',
        'container_ports_raw',
        'container_ports',
        'container_mounts',
        'container_labels_raw',
        'container_local_names',
        'container_size'
    ]

    # Add dynamically label_* fields (if any...)
    extra_cols = sorted({c for row in expanded_rows for c in row.keys()} - set(cols))
    columns = cols + extra_cols

    return pd.DataFrame(expanded_rows, columns=columns)

def merge_network_and_container_data(network_df: pd.DataFrame,
                                     container_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge expand_network_data() and  expand_container_data()
    by container_id.
    """

    # Standardize key
    network_df['container_id'] = network_df['container_id'].fillna('').astype(str).str[:12]
    container_df['container_id'] = container_df['container_id'].fillna('').astype(str).str[:12]

    #  merge
    merged = pd.merge(
        network_df,
        container_df,
        on='container_id',
        how='left',
        suffixes=('_network', '_container')
    )

    return merged


In [ ]:
# Read data and process initial fill

containers_df =pd.read_json('containers-inv.json')
containers_df = expand_container_data(containers_df)

networks1_df=pd.read_json('network1.json')
networks1_df = expand_network_data(networks1_df)

networks2_df=pd.read_json('network2.json')
networks2_df = expand_network_data(networks2_df)

nwk_df=pd.concat([networks1_df,
                  networks2_df,])


In [ ]:
# merge all in one
merged_df = merge_network_and_container_data(nwk_df, contaibners_df)

In [ ]:
# add extra fields

merged_df['container_description']=''
merged_df['container_ncpu']=''
merged_df['container_nram']=''
merged_df['container_ndisk']=''
merged_df['container_hostname']=''

In [ ]:
# check structure
merged_df.info()